### Instalando o Experta

In [1]:
!pip install experta

### Importando dependencias

In [2]:
from experta import *

### Funções uteis

In [39]:
# Funçao auxiliar para receber inputs do teclado
def ask(question, true_value=True, false_value=False):
    while True:
        answer = input(question + "(S/N)").upper()
        if (answer == 'S'): return true_value
        if (answer == 'N'): return false_value
        print("Responda com 'S' para Sim ou 'N' para Não.")

def ask_feature(question, true_value=True, false_value=False):
    return ask("O animal " + question, true_value, false_value)

# EXEMPLO
print(ask_feature("amamenta?", "mamífero", "não é mamífero"))

mamífero


In [98]:
# Definindo tipos de fatos do dominio
class Classe(Fact):
    pass

class Feature(Fact):
    pass

class Animal(Fact):
    pass

# Aliases de prioridade
HIGHEST = 3
HIGH = 2 
MEDIUM = 1
DEFAULT = 0 # non-needed
LOWEST = -1

In [110]:
class DefineAnimal(KnowledgeEngine):
    # @DefFacts()
    # def _initial_action(self):
    #     yield Classe('Réptil')

    # FEATURES PARA DEFINIR CLASSE

    @Rule(NOT(Classe(W())))
    def ask_amamenta(self):
        self.declare(Feature(amamenta=ask_feature("é amamentado?")))

    @Rule(NOT(Classe(W())))
    def ask_branqueas(self):
        self.declare(Feature(branqueas=ask_feature("possui brânqueas (Guelras)?")))
    
    @Rule(NOT(Classe(W())))
    def ask_ovo_casca(self):
        self.declare(Feature(ovo_casca=ask_feature("bota ovos com casca rígida?")))
    
    @Rule(NOT(Classe(W())))
    def ask_duas_fases(self):
        self.declare(Feature(duas_fases=ask_feature("passa o começo da vida na água, e depois vive na terra?")))

    @Rule(NOT(Classe(W())), salience=HIGH)
    def ask_possui_penas(self):
        self.declare(Feature(possui_penas=ask_feature("possui penas?")))

    # DEFININDO CLASSE

    @Rule(Feature(amamenta=True), salience=HIGHEST)
    def is_mamal(self):
        self.declare(Classe('Mamífero'))

    @Rule(Feature(branqueas=True), salience=HIGHEST)
    def is_fish(self):
        self.declare(Classe('Peixe'))
    
    @Rule(Feature(duas_fases=True), salience=HIGHEST)
    def is_amphibian(self):
        self.declare(Classe('Anfibio'))
    
    @Rule(Feature(possui_penas=True),salience=HIGHEST)
    def is_aves(self):
        self.declare(Classe("Ave"))
    
    @Rule(AND(Feature(ovo_casca=True),Feature(possui_penas=False)),salience=HIGHEST)
    def is_repitile(self):
        self.declare(Classe("Réptil"))
        

 
    # COLETANDO FEATURES PARA DEFINIR O ANIMAL:

    @Rule(AND( OR(Classe("Ave"), Classe("Réptil"), Classe("Mamífero")), 
               NOT(Animal(W())) ))
    def ask_carnivoro(self):
        self.declare(Feature(carnivoro=ask_feature("é carnívoro?")))

    @Rule(AND( OR(Classe("Ave"), Classe("Réptil"), Classe("Mamífero")),
               NOT(Animal(W())) ))
    def ask_aquatico_ou_semi(self):
        self.declare(Feature(aquatico_ou_semi=ask_feature("é aquatico ou precisa de viver perto da água?")))

    @Rule(AND( OR(Classe("Ave"), Classe("Mamífero")),
               NOT(Animal(W())) ))
    def ask_voa(self):
        self.declare(Feature(voa=ask_feature("é capaz de voar?")))

    @Rule(AND( Classe("Mamífero"), 
               NOT(Animal(W())) ))
    def ask_selvagem(self):
        self.declare(Feature(selvagem=ask_feature("é selvagem?")))

    @Rule(AND( Classe("Peixe"), 
               NOT(Animal(W())) ))
    def ask_cartilaginoso(self):
        self.declare(Feature(cartilaginoso=ask_feature("possui esqueleto cartilaginoso?")))
    
    @Rule(AND( Classe("Peixe"), 
               NOT(Animal(W())) ))
    def ask_fusiforme(self):
        self.declare(Feature(fusiforme=ask_feature("tem forma alongada (fusiforme)?")))

    @Rule(AND( Classe("Anfibio"), 
               NOT(Animal(W())) ))
    def ask_rabo(self):
        self.declare(Feature(rabo=ask_feature("possui rabo?")))

    # DIFERENCIANDO RÉPTEIS

    @Rule(AND(Classe("Réptil"), Feature(carnivoro=True), Feature(aquatico_ou_semi=True)), salience=HIGHEST)
    def is_jacare(self):
        self.declare(Animal("Jacaré"))
    
    @Rule(AND(Classe("Réptil"), Feature(carnivoro=True), Feature(aquatico_ou_semi=False)), salience=HIGHEST)
    def is_cobra(self):
        self.declare(Animal("Cobra"))
    
    @Rule(AND(Classe("Réptil"), Feature(carnivoro=False), Feature(aquatico_ou_semi=True)), salience=HIGHEST)
    def is_tartaruga(self):
        self.declare(Animal("Tartaruga"))

    @Rule(AND(Classe("Réptil"), Feature(carnivoro=False), Feature(aquatico_ou_semi=False)), salience=HIGHEST)
    def is_camaleao(self):
        self.declare(Animal("Camaleão"))

    # DIFERENCIANDO PEIXES

    @Rule(AND(Classe("Peixe"), Feature(cartilaginoso=True), Feature(fusiforme=True)), salience=HIGHEST)
    def is_tubarao(self):
        self.declare(Animal("Tubarão"))
    
    @Rule(AND(Classe("Peixe"), Feature(cartilaginoso=True), Feature(fusiforme=False)), salience=HIGHEST)
    def is_arraia(self):
        self.declare(Animal("Arraia"))

    @Rule(AND(Classe("Peixe"), Feature(cartilaginoso=False), Feature(fusiforme=True)), salience=HIGHEST)
    def is_atum(self):
        self.declare(Animal("Atum"))
    
    @Rule(AND(Classe("Peixe"), Feature(cartilaginoso=False), Feature(fusiforme=False)), salience=HIGHEST)
    def is_baiacu(self):
        self.declare(Animal("Baiacu"))

    # DIFERENCIANDO AVES

    @Rule(AND(Classe("Ave"), Feature(voa=False)), salience=HIGHEST)
    def is_pinguim(self):
        self.declare(Animal("Pinguim"))
    
    @Rule(AND(Classe("Ave"), Feature(carnivoro=False)), salience=HIGHEST)
    def is_beija_flor(self):
        self.declare(Animal("Beija-flor"))

    @Rule(AND(Classe("Ave"), Feature(aquatico_ou_semi=True), Feature(voa=True)), salience=HIGHEST)
    def is_gaivota(self):
        self.declare(Animal("Gaivota"))

    @Rule(AND(Classe("Ave"), Feature(aquatico_ou_semi=False), Feature(carnivoro=True)), salience=HIGHEST)
    def is_gaviao(self):
        self.declare(Animal("Gavião"))

    # DIFERENCIANDO MAMIFEROS

    @Rule(AND(Classe("Mamífero"), Feature(voa=True)), salience=HIGHEST)
    def is_morcego(self):
        self.declare(Animal("Morcego"))
    
    @Rule(AND(Classe("Mamífero"), Feature(aquatico_ou_semi=True)), salience=HIGHEST)
    def is_baleia(self):
        self.declare(Animal("Baleia"))

    @Rule(AND(Classe("Mamífero"), Feature(selvagem=False), Feature(carnivoro=False)), salience=HIGHEST)
    def is_humano(self):
        self.declare(Animal("Humano"))
    
    @Rule(AND(Classe("Mamífero"), Feature(selvagem=False), Feature(carnivoro=True)), salience=HIGHEST)
    def is_cachorro(self):
        self.declare(Animal("Cachorro"))

    @Rule(AND(Classe("Mamífero"), 
            Feature(aquatico_ou_semi=False), 
            Feature(selvagem=True), 
            Feature(carnivoro=True)), 
        salience=HIGHEST)
    def is_urso(self):
        self.declare(Animal("Urso"))

    # DIFERENCIANDO ANFIBIOS

    @Rule(AND(Classe('Anfíbio'), Feature(rabo=True)), salience=HIGHEST)
    def is_salamandra(self):
        self.declare(Animal('Salamandra'))

    @Rule(AND(Classe('Anfíbio'), Feature(rabo=False)), salience=HIGHEST)
    def is_sapo(self):
        self.declare(Animal('Sapo'))


    # FAZENDO MATCH COM O ANIMAL DESCOBERTO
    @Rule(Animal(MATCH.name))
    def guess_animal(self, name):
        print("O seu animal é um(a) %s" % (name))




In [112]:
engine = DefineAnimal()
run = ask("Olá! Vamos identificar um animal?")
while(run):
    engine.reset()
    engine.run()
    engine.facts
    run = ask("Deseja identificar outro animal?")

O seu animal é um(a) Urso
O seu animal é um(a) Cachorro
O seu animal é um(a) Humano
O seu animal é um(a) Humano


In [115]:
class DefineClasseAnimal(KnowledgeEngine):
    # FEATURES PARA DEFINIR CLASSE

    @Rule(NOT(Classe(W())))
    def ask_amamenta(self):
        self.declare(Feature(amamenta=ask_feature("é amamentado?")))

    @Rule(NOT(Classe(W())))
    def ask_branqueas(self):
        self.declare(Feature(branqueas=ask_feature("possui brânqueas (Guelras)?")))
    
    @Rule(NOT(Classe(W())))
    def ask_ovo_casca(self):
        self.declare(Feature(ovo_casca=ask_feature("bota ovos com casca rígida?")))
    
    @Rule(NOT(Classe(W())))
    def ask_duas_fases(self):
        self.declare(Feature(duas_fases=ask_feature("passa o começo da vida na água, e depois vive na terra?")))

    @Rule(NOT(Classe(W())), salience=HIGH)
    def ask_possui_penas(self):
        self.declare(Feature(possui_penas=ask_feature("possui penas?")))

    # DEFININDO CLASSE

    @Rule(Feature(amamenta=True), salience=HIGHEST)
    def is_mamal(self):
        self.declare(Classe('Mamífero'))

    @Rule(Feature(branqueas=True), salience=HIGHEST)
    def is_fish(self):
        self.declare(Classe('Peixe'))
    
    @Rule(Feature(duas_fases=True), salience=HIGHEST)
    def is_amphibian(self):
        self.declare(Classe('Anfibio'))
    
    @Rule(Feature(possui_penas=True),salience=HIGHEST)
    def is_aves(self):
        self.declare(Classe("Ave"))
    
    @Rule(AND(Feature(ovo_casca=True),Feature(possui_penas=False)),salience=HIGHEST)
    def is_repitile(self):
        self.declare(Classe("Réptil"))

In [116]:
class DefineAnimal(DefineClasseAnimal):
    # COLETANDO FEATURES PARA DEFINIR O ANIMAL:

    @Rule(AND( OR(Classe("Ave"), Classe("Réptil"), Classe("Mamífero")), 
               NOT(Animal(W())) ))
    def ask_carnivoro(self):
        self.declare(Feature(carnivoro=ask_feature("é carnívoro?")))

    @Rule(AND( OR(Classe("Ave"), Classe("Réptil"), Classe("Mamífero")),
               NOT(Animal(W())) ))
    def ask_aquatico_ou_semi(self):
        self.declare(Feature(aquatico_ou_semi=ask_feature("é aquatico ou precisa de viver perto da água?")))

    @Rule(AND( OR(Classe("Ave"), Classe("Mamífero")),
               NOT(Animal(W())) ))
    def ask_voa(self):
        self.declare(Feature(voa=ask_feature("é capaz de voar?")))

    @Rule(AND( Classe("Mamífero"), 
               NOT(Animal(W())) ))
    def ask_selvagem(self):
        self.declare(Feature(selvagem=ask_feature("é selvagem?")))

    @Rule(AND( Classe("Peixe"), 
               NOT(Animal(W())) ))
    def ask_cartilaginoso(self):
        self.declare(Feature(cartilaginoso=ask_feature("possui esqueleto cartilaginoso?")))
    
    @Rule(AND( Classe("Peixe"), 
               NOT(Animal(W())) ))
    def ask_fusiforme(self):
        self.declare(Feature(fusiforme=ask_feature("tem forma alongada (fusiforme)?")))

    @Rule(AND( Classe("Anfibio"), 
               NOT(Animal(W())) ))
    def ask_rabo(self):
        self.declare(Feature(rabo=ask_feature("possui rabo?")))

    # DIFERENCIANDO RÉPTEIS

    @Rule(AND(Classe("Réptil"), Feature(carnivoro=True), Feature(aquatico_ou_semi=True)), salience=HIGHEST)
    def is_jacare(self):
        self.declare(Animal("Jacaré"))
    
    @Rule(AND(Classe("Réptil"), Feature(carnivoro=True), Feature(aquatico_ou_semi=False)), salience=HIGHEST)
    def is_cobra(self):
        self.declare(Animal("Cobra"))
    
    @Rule(AND(Classe("Réptil"), Feature(carnivoro=False), Feature(aquatico_ou_semi=True)), salience=HIGHEST)
    def is_tartaruga(self):
        self.declare(Animal("Tartaruga"))

    @Rule(AND(Classe("Réptil"), Feature(carnivoro=False), Feature(aquatico_ou_semi=False)), salience=HIGHEST)
    def is_camaleao(self):
        self.declare(Animal("Camaleão"))

    # DIFERENCIANDO PEIXES

    @Rule(AND(Classe("Peixe"), Feature(cartilaginoso=True), Feature(fusiforme=True)), salience=HIGHEST)
    def is_tubarao(self):
        self.declare(Animal("Tubarão"))
    
    @Rule(AND(Classe("Peixe"), Feature(cartilaginoso=True), Feature(fusiforme=False)), salience=HIGHEST)
    def is_arraia(self):
        self.declare(Animal("Arraia"))

    @Rule(AND(Classe("Peixe"), Feature(cartilaginoso=False), Feature(fusiforme=True)), salience=HIGHEST)
    def is_atum(self):
        self.declare(Animal("Atum"))
    
    @Rule(AND(Classe("Peixe"), Feature(cartilaginoso=False), Feature(fusiforme=False)), salience=HIGHEST)
    def is_baiacu(self):
        self.declare(Animal("Baiacu"))

    # DIFERENCIANDO AVES

    @Rule(AND(Classe("Ave"), Feature(voa=False)), salience=HIGHEST)
    def is_pinguim(self):
        self.declare(Animal("Pinguim"))
    
    @Rule(AND(Classe("Ave"), Feature(carnivoro=False)), salience=HIGHEST)
    def is_beija_flor(self):
        self.declare(Animal("Beija-flor"))

    @Rule(AND(Classe("Ave"), Feature(aquatico_ou_semi=True), Feature(voa=True)), salience=HIGHEST)
    def is_gaivota(self):
        self.declare(Animal("Gaivota"))

    @Rule(AND(Classe("Ave"), Feature(aquatico_ou_semi=False), Feature(carnivoro=True)), salience=HIGHEST)
    def is_gaviao(self):
        self.declare(Animal("Gavião"))

    # DIFERENCIANDO MAMIFEROS

    @Rule(AND(Classe("Mamífero"), Feature(voa=True)), salience=HIGHEST)
    def is_morcego(self):
        self.declare(Animal("Morcego"))
    
    @Rule(AND(Classe("Mamífero"), Feature(aquatico_ou_semi=True)), salience=HIGHEST)
    def is_baleia(self):
        self.declare(Animal("Baleia"))

    @Rule(AND(Classe("Mamífero"), Feature(selvagem=False), Feature(carnivoro=False)), salience=HIGHEST)
    def is_humano(self):
        self.declare(Animal("Humano"))
    
    @Rule(AND(Classe("Mamífero"), Feature(selvagem=False), Feature(carnivoro=True)), salience=HIGHEST)
    def is_cachorro(self):
        self.declare(Animal("Cachorro"))

    @Rule(AND(Classe("Mamífero"), 
            Feature(aquatico_ou_semi=False), 
            Feature(selvagem=True), 
            Feature(carnivoro=True)), 
        salience=HIGHEST)
    def is_urso(self):
        self.declare(Animal("Urso"))

    # DIFERENCIANDO ANFIBIOS

    @Rule(AND(Classe('Anfíbio'), Feature(rabo=True)), salience=HIGHEST)
    def is_salamandra(self):
        self.declare(Animal('Salamandra'))

    @Rule(AND(Classe('Anfíbio'), Feature(rabo=False)), salience=HIGHEST)
    def is_sapo(self):
        self.declare(Animal('Sapo'))


    # FAZENDO MATCH COM O ANIMAL DESCOBERTO
    @Rule(Animal(MATCH.name))
    def guess_animal(self, name):
        print("O seu animal é um(a) %s" % (name))

In [ ]:
engine = DefineAnimal()
run = ask("Olá! Vamos identificar um animal?")
while(run):
    engine.reset()
    engine.run()
    engine.facts
    run = ask("Deseja identificar outro animal?")